## SQL Practice


In this lab, you're going to have ten prompts. Each prompt will require you to do two things:
1. Write a SQL query.
2. Use the result of that query to answer the question.

**Data**: The data used in this lab is the famous (but fake) Northwind database. It contains various tables of transactions. The schema for the Northwind database can be found [here](./assets/Northwind_ERD.png).

**Important**: The table and label names in the SQLite database are **slightly different than the ERD**. In particular, many of the tables names are singular where the ERD lists them as plural, e.g. the `Customers` table in the ERD is actually called `Customer` in the SQLite database. The instructions should provide the correct table names. The field names themselves should be correct.

You'll be using Python's `sqlite3` library this time. `sqlite3` comes with Python, so you shouldn't have to install anything. You will once again be passing queries into `pd.read_sql_query()` as strings. If you try to close the notebook and get an alert that there is still a connection to the database, you can run `con.close()` to close the connection. (Alternatively, it should be safe to force exit.)

In [1]:
import pandas as pd
from pandas.io import sql

import sqlite3

### Prompt 1

How many rows are there in the `Customer` table of the Northwind database?

In [2]:
con = sqlite3.connect('data/Northwind_small.sqlite')

sql = """
SELECT COUNT(*) AS num_rows
FROM Customer
AS customer_count;
"""

pd.read_sql_query(sql, con)

,num_rows
0,91


**Prompt 1 Answer**: There are 91 rows in the customers table of the Northwind database

### Prompt 2

If you sort the customers in alphabetical order by name, which customer is fourth?

(You can answer this by retrieving the first four customers; you _do not_ need to retrieve only the 4th customer when customers are sorted alphabetically.)

In [3]:
con = sqlite3.connect('data/Northwind_small.sqlite')

sql = """
SELECT ContactName FROM Customer
ORDER BY ContactName 
LIMIT 4
"""

pd.read_sql_query(sql, con)

,ContactName
0,Alejandra Camino
1,Alexander Feuer
2,Ana Trujillo
3,Anabela Domingues


**Prompt 2 Answer**: Anabela Domingues is fourth when you sort the customers in alphabetical order by name



### Prompt 3

Among those customers who live in Germany, if you sort the customers in alphabetical order by name, which customer is fourth?

In [4]:
# Use a SQL query to find the answer to the above prompt.
con = sqlite3.connect('data/Northwind_small.sqlite')

sql = """
SELECT ContactName FROM Customer
WHERE Country = 'Germany'
ORDER BY ContactName 
LIMIT 4;
"""

pd.read_sql_query(sql, con)

,ContactName
0,Alexander Feuer
1,Hanna Moos
2,Henriette Pfalzheim
3,Horst Kloss


**Prompt 3 Answer**: Horst Kloss is fourth when you sort the German customers in alphabetical order by name




### Prompt 4

From the `Employee` table, what are the first names of the individuals who have a birthdate in the 1960s?

In [5]:
# Use a SQL query to find the answer to the above prompt.
con = sqlite3.connect('data/Northwind_small.sqlite')

sql = """
SELECT FirstName
FROM Employee
WHERE BirthDate BETWEEN '1960-01-01' AND '1969-12-31';
"""

pd.read_sql_query(sql, con)

,FirstName
0,Margaret


**Prompt 4 Answer**: Just Margaret.

### Prompt 5

In `OrderDetail` table of the Northwind database, what is the total number of units sold for products 2, 3, and 4? 

You should give us three numbers - one for each `ProductID`.

> You can solve this with either three separate queries, or with just one query.

In [6]:
# Use a SQL query to find the answer to the above prompt.
con = sqlite3.connect('data/Northwind_small.sqlite')

sql = """
SELECT ProductId, SUM(Quantity) AS total_units_sold
FROM OrderDetail
WHERE ProductId IN (2, 3, 4) 
GROUP BY ProductId;
"""

pd.read_sql_query(sql, con)

,ProductId,total_units_sold
0,2,1057
1,3,328
2,4,453


**Prompt 5 Answer**: The total number of units sold for products 2, 3, and 4 are 1057, 328, and 453 (respectively)

### Prompt 6

From the `OrderDetail` table of the Northwind database, calculate the revenue per product. In your SQL output, rename the revenue column `Revenue` and sort your results from largest revenue per product to smallest revenue per product. Which `ProductID` has the fifth-highest revenue, and what is that revenue rounded to the nearest dollar?

Note that revenue for a given product should be equal to the total unit price of that product times the quantity. You do not need to worry about the discount here. If you haven't multiplied columns together before, [this StackOverflow question](https://stackoverflow.com/questions/7536996/multiplying-two-columns-in-sql-server/7537059) may be helpful.

In [7]:
# Use a SQL query to find the answer to the above prompt.
con = sqlite3.connect('data/Northwind_small.sqlite')

sql = """
SELECT ProductID, SUM(UnitPrice * Quantity) AS Revenue
FROM OrderDetail
GROUP BY ProductID
ORDER BY Revenue DESC
LIMIT 5;
"""

pd.read_sql_query(sql, con)

,ProductId,Revenue
0,38,149984.2
1,29,87736.4
2,59,76296.0
3,60,50286.0
4,62,49827.9


**Prompt 6 Answer**:  ProductID = 62 has the fifth-highest revenue, which rounds to $49,828.

### Prompt 7

From the `Product` table of the Northwind database, show the `ProductName`, `UnitsInStock`, and `UnitsOnOrder` values for those products who have more units on order than units in stock. Among these results, which product has the most units on order?

In [8]:
# Use a SQL query to find the answer to the above prompt.
con = sqlite3.connect('data/Northwind_small.sqlite')

sql = """
SELECT
    ProductName, UnitsInStock, UnitsOnOrder
FROM Product
WHERE UnitsOnOrder > UnitsInStock
ORDER BY UnitsOnOrder DESC
LIMIT 1;
"""

pd.read_sql_query(sql, con)

,ProductName,UnitsInStock,UnitsOnOrder
0,Louisiana Hot Spiced Okra,4,100


**Prompt 7 Answer**: Louisiana Hot Spiced Okra has the most units on order

### Prompt 8

From the `OrderDetail` table, show the products that have an average discount of at least 8%. Among these results, which average discount is closest to 8%?

In [9]:
# Use a SQL query to find the answer to the above prompt.
con = sqlite3.connect('data/Northwind_small.sqlite')

sql = """
SELECT
    ProductId,
    AVG(Discount) AS avg_discount
FROM OrderDetail
GROUP BY ProductId
HAVING avg_discount >= 0.08
ORDER BY avg_discount ASC
"""

pd.read_sql_query(sql, con)

,ProductId,avg_discount
0,30,0.081250
1,25,0.086111
2,9,0.100000
3,58,0.100000
4,2,0.102273
5,48,0.108333


**Prompt 8 Answer**: Product_ID = 30 had the average discount closest to 8% 

### Prompt 9

Return the `Orders` table, but also include the first name, last name, and country of the employee assigned to each order (you'll need the `Employee` table for this.) What is the first and last name of the employee assigned to OrderID 11077?

In [10]:
con = sqlite3.connect('data/Northwind_small.sqlite')

sql = """
SELECT FirstName, LastName, o.Id
FROM Orders AS o
LEFT JOIN Employee AS e
ON e.Id = o.EmployeeID
WHERE o.Id = 11077;
"""

pd.read_sql_query(sql, con)

,FirstName,LastName,Id
0,Nancy,Davolio,11077


**Prompt 9 Answer**: Nancy Davolio is first and last name of the employee assigned to order_id 11077. 

### Prompt 10

Find the average amount of freight across all orders, broken out by country of *employee*. (**NOT `ShipCountry`**.) How much higher is the average freight from UK employees than the average freight from US employees?

> For a bonus, round the freight to the nearest whole number and rename the column `Average Freight`.

In [11]:
con = sqlite3.connect('data/Northwind_small.sqlite')

sql = """
SELECT e.Country, AVG(Freight)
FROM Orders AS o
LEFT JOIN Employee AS e
ON o.EmployeeId = e.Id
GROUP BY e.Country;
"""

pd.read_sql_query(sql, con)

,Country,AVG(Freight)
0,UK,78.977143
1,USA,77.973284


**Prompt 10 Answer**: UK employees' average freight is 1.003859 higher than the average freight from US employees. 

Once you're done, run the following cell to close your connection to the database:

In [12]:
con.close()